# Plan
- inputs
  - engine
  - prompt
  - conversation
  - good responses
  - bad responses
- flow
  - run a chat completion on engine, prompt, and conversation
  - embed output from completion
  - compare output embedding to good and bad

In [26]:
import openai
import numpy as np

In [2]:
client = openai.OpenAI()

In [6]:
resp = client.chat.completions.create(model='gpt-4o-mini', messages=[{'role': 'user', 'content': 'hi'}])

In [7]:
resp

ChatCompletion(id='chatcmpl-AAhEX3SWpGAcHgti2m6fWw2W7PmgZ', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello! How can I assist you today?', role='assistant', function_call=None, tool_calls=None, refusal=None))], created=1727112833, model='gpt-4o-mini-2024-07-18', object='chat.completion', system_fingerprint='fp_1bb46167f9', usage=CompletionUsage(completion_tokens=9, prompt_tokens=8, total_tokens=17, completion_tokens_details={'reasoning_tokens': 0}))

In [11]:
resp.choices[0].message.content

'Hello! How can I assist you today?'

In [55]:
def get_response(model, prompt, conversation) -> str:
    messages = [
        {'role': 'system', 'content': prompt},
        *conversation
    ]
    resp = client.chat.completions.create(model=model, messages=messages)
    return resp.choices[0].message.content

In [56]:
get_response(
    'gpt-4o-mini',
    'Speak like a Texan.',
    [{'role': 'user', 'content': 'hi'}]
)

"Howdy, partner! What brings ya 'round these parts today?"

In [34]:
good = 'Howdy! What can I do for you?'
bad = 'Hello! How can I assist you?'

In [35]:
resp = client.embeddings.create(input=[good, bad], model='text-embedding-3-small')

In [36]:
embeds = np.array([emb.embedding for emb in resp.data])

In [37]:
embeds

array([[-0.01905102, -0.02874077, -0.02385484, ..., -0.03446155,
         0.00050083,  0.01029878],
       [-0.00424917, -0.0104074 ,  0.02297701, ..., -0.00539529,
         0.01098216,  0.00192958]])

In [38]:
embeds.shape

(2, 1536)

In [44]:
def get_embedding(text) -> np.array:
    resp = client.embeddings.create(input=text, model='text-embedding-3-small')
    return np.array(resp.data[0].embedding).reshape(1,-1)

In [47]:
embed = get_embedding('Hello! How can I assist you today?')

In [48]:
embed @ embeds.T

array([[0.63589733, 0.94381284]])

In [54]:
get_response('gpt-4o', 'Speak in spanish, regardless of the user\'s language', [{'role':'user', 'content':'hi'}])

'Hello! How can I assist you today?'

In [57]:
reply = get_response(
    'gpt-4o-mini',
    'Speak like a Texan.',
    [{'role': 'user', 'content': 'hi'}]
)
print(reply)
embed = get_embedding(reply)
embed @ embeds.T

Howdy, partner! How’s it goin’ with ya today?


array([[0.63786166, 0.40458722]])